In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

2023-01-21 11:09:45.528982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 11:09:55.934219: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-21 11:09:55.934552: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-21 11:09:57.802566: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-21 11:10:06.878985: W tensorflow/stream_executor/platform/de

'2.10.0'

In [2]:
path = 'test/rose/118974357_0faa23cce9_n.jpg'

In [3]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [4]:
model = keras.models.load_model('model_xception.h5')

2023-01-21 11:10:34.653815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-21 11:10:34.654141: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-21 11:10:34.654320: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-21 11:10:34.654496: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-21 11:10:34.654638: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [5]:
img = load_img(path, target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [6]:
preds = model.predict(X)
preds

1/1 [==============================] - 6s 6s/step


array([[-5.004658 , -2.320822 ,  4.7676196, -4.0044637, -2.9362733]],
      dtype=float32)

In [7]:
classes = [
    'daisy',
    'dandelion',
    'rose',
    'sunflower',
    'tulip'
]

In [8]:
dict(zip(classes, preds[0]))

{'daisy': -5.004658,
 'dandelion': -2.320822,
 'rose': 4.7676196,
 'sunflower': -4.0044637,
 'tulip': -2.9362733}

## Convert Keras to tf.lite

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('flowers_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpby0rg83_/assets


INFO:tensorflow:Assets written to: /tmp/tmpby0rg83_/assets
2023-01-08 20:22:12.455821: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-08 20:22:12.456298: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-08 20:22:12.464883: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpby0rg83_
2023-01-08 20:22:12.522285: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-08 20:22:12.524616: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpby0rg83_
2023-01-08 20:22:12.686571: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-01-08 20:22:12.736943: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-08 20:22:15.670585: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [10]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path='flowers_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [12]:
classes = [
    'daisy',
    'dandelion',
    'rose',
    'sunflower',
    'tulip'
]

dict(zip(classes, preds[0]))

{'daisy': -5.0046597,
 'dandelion': -2.3208249,
 'rose': 4.7676163,
 'sunflower': -4.0044613,
 'tulip': -2.9362707}

## Removing TF dependency

In [13]:
from PIL import Image
with Image.open(path) as img:
    img = img.resize((299, 299), Image.NEAREST)

In [14]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.0
    return x
    
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [15]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [16]:
classes = [
    'daisy',
    'dandelion',
    'rose',
    'sunflower',
    'tulip'
]

dict(zip(classes, preds[0]))

{'daisy': -5.0046597,
 'dandelion': -2.3208249,
 'rose': 4.7676163,
 'sunflower': -4.0044613,
 'tulip': -2.9362707}

In [18]:
#!pip install keras-image-helper

In [27]:
#!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


## Simpler way of doing it

In [1]:
from keras_image_helper import create_preprocessor
#import tensorflow.lite as tflite
#change tensorflow with tflite_runtime
import tflite_runtime.interpreter as tflite

In [2]:
interpreter = tflite.Interpreter(model_path='flowers_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
preprocessor = create_preprocessor('xception', target_size = (299,299))

In [4]:
url = 'https://www.thespruce.com/thmb/V9sZh_5_x4UwS1BRGmo3TjH7n-c=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/daisy-types-for-gardens-1316051-04-e0f4e84f081d452885752e84bc7886ed.jpg'
X = preprocessor.from_url(url)
# also can use preprocessor.from_path

In [5]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [6]:
classes = [
    'daisy',
    'dandelion',
    'rose',
    'sunflower',
    'tulip'
]

dict(zip(classes, preds[0]))

{'daisy': 7.6587334,
 'dandelion': 2.7163494,
 'rose': -2.0467002,
 'sunflower': -2.7290316,
 'tulip': -5.8923254}